In [3]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0glqrxeh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0glqrxeh
  Resolved https://github.com/huggingface/transformers to commit baa765f813942d7ba7d8e58ed2f8ca01d826afed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.4 MB/s eta 0:00:0000:0100:01
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9924470 sha256=50a69448bdf23f08f76901420f35871b5af7238ecd79da545784563d298e2966
  Stored in directory: /tmp/pip-ephem-wheel-cache-wuz48won/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Succ

In [4]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, AutoTokenizer

In [6]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype = "auto",
    device_map = "auto"
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [7]:
from PIL import Image, ImageEnhance
import torch
import requests

def compress_and_enhance_image(input_image, max_size=(1024, 1024), quality=85):
    # Open and compress the image
    img = input_image.copy()
    
    # Resize the image to fit within the max dimensions
    img.thumbnail(max_size)
    
    # Convert to grayscale to enhance text contrast for OCR
    img = img.convert("L")
    
    # Increase contrast for better OCR accuracy
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2.0)  # Adjust contrast factor as needed
    
    # Save or use the image in memory
    return img

def process_image(img):
    
    # Load image (from your provided path or another method)
    image = Image.open(img)

    # Compress and enhance the image for OCR
    enhanced_image = compress_and_enhance_image(image)

    
    # Existing code for processing the image with the model
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image"
                },
                {
                    "type": "text",
                    "text": "Extract all text from image."
                }
            ]
        }
    ]

    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    inputs = processor(
        text = [text_prompt],
        images = [enhanced_image],  # Use the enhanced and compressed image
        padding = True,
        return_tensors = "pt"
    )

    inputs = inputs.to("cuda")

    output_ids = model.generate(**inputs, max_new_tokens=1024)

    generated_ids = [
        output_ids[len(input_ids) : ]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Output extracted text
    return output_text


In [8]:
process_image("/kaggle/input/sample/test.jpeg")

['Here is the extracted text from the image:\n\nAbout - के बारे में\nAbout me - मेरे बारे में\nAbout us - एक्स मेरे बारे में\nAbout them - उनके बारे में\nAbout you - आपके बारे में\nAbout him - उनके बारे में\nAbout it - इसके बारे में\nAbout whom - जिसके बारे में\nAbout now - अभी के बारे में\nAbout today - आज के बारे में\nAbout life - जीवन के बारे में\nAbout food - खाने के बारे में\nAbout here - यहाँ के बारे में\nAbout there - यहाँ के बारे में\nAbout love - आंतर के बारे में']